In [19]:
from models.id_to_party_dict import id_to_party_dict
import numpy as np
import pandas as pd
import psycopg2
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, silhouette_score
import matplotlib.pyplot as plt
import time

start_time = time.time()


# Paramètres de connexion à la base de données PostgreSQL
conn_params = {
    "host": "localhost",
    "port": "15432",
    "database": "postgres",
    "user": "admin",
    "password": "admin"
}

# Connexion à la base de données
conn = psycopg2.connect(**conn_params)

# Requêtes SQL pour récupérer les données nécessaires
query_demographie = "SELECT * FROM demographie;"
query_economie = "SELECT * FROM economie;"
query_election = "SELECT * FROM election_2022_t1;"

# query_election = """
#     SELECT "Parti 1", "% Voix/Exp 1", "Parti 2", "% Voix/Exp 2", "Parti 3", "% Voix/Exp 3", 
#             "Parti 4", "% Voix/Exp 4", "Parti 5", "% Voix/Exp 5", "Parti 6", "% Voix/Exp 6", 
#             "Parti 7", "% Voix/Exp 7", "Parti 8", "% Voix/Exp 8", "Parti 9", "% Voix/Exp 9", 
#             "Parti 10", "% Voix/Exp 10", "Parti 11", "% Voix/Exp 11", "Parti 12", "% Voix/Exp 12"
#      FROM election_2022_t1
# """
# df_election = pd.read_sql(query_election, conn)

# Chargement des données
df_demographie = pd.read_sql(query_demographie, conn)
df_economie = pd.read_sql(query_economie, conn)
df_election = pd.read_sql(query_election, conn)

# print(df_demographie.columns)
# print(df_economie.columns)
# print(df_election.columns)

# Fermeture de la connexion à la base de données
conn.close()

# Mapper les noms des partis politiques aux identifiants numériques dans df_election

party_to_id_dict = {v: k for k, v in id_to_party_dict.items()}

# Mapper les noms des partis politiques aux identifiants numériques dans df_election
for i in range(1, 13):
    df_election[f"Parti {i}"] = df_election[f"Parti {i}"].map(party_to_id_dict)

# Fusion des DataFrames
df_merged = pd.merge(pd.merge(df_demographie, df_economie, on='code_postal'), df_election, on='code_postal')

# Sélection des caractéristiques pour le clustering
features_to_include = [
    '65-99_2020', '60-99_2023', '25-59_2023', '0-24_2023',
    'dens_pop', 'avg_2023', 'Part des minima sociaux dans le rev. disp. 2021',
    'Part des prestations familiales dans le rev. disp. 2021', 'Médiane du niveau de vie 2021',
    '% Voix/Ins 1', '% Voix/Ins 2', '% Voix/Ins 3', '% Voix/Ins 4', '% Voix/Ins 5',
    '% Voix/Ins 6', '% Voix/Ins 7', '% Voix/Ins 8', '% Voix/Ins 9',
    '% Voix/Ins 10', '% Voix/Ins 11', '% Voix/Ins 12', '% Voix/Exp 1',
    '% Voix/Exp 2', '% Voix/Exp 3', '% Voix/Exp 4', '% Voix/Exp 5',
    '% Voix/Exp 6', '% Voix/Exp 7', '% Voix/Exp 8', '% Voix/Exp 9',
    '% Voix/Exp 10', '% Voix/Exp 11', '% Voix/Exp 12'
]

# Imputation des valeurs manquantes et standardisation
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(df_merged[features_to_include])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Réduction de dimension avec PCA
pca = PCA(n_components=0.95)  # Conserver 95% de la variance expliquée
X_pca = pca.fit_transform(X_scaled)

# Optimisation du nombre de clusters avec le score de silhouette
silhouette_scores = []
range_clusters = range(2, 12)
for n_clusters in range_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(X_pca)
    silhouette_avg = silhouette_score(X_pca, cluster_labels)
    silhouette_scores.append(silhouette_avg)

# Affichage du score de silhouette pour chaque nombre de clusters
plt.figure(figsize=(10, 6))
plt.plot(range_clusters, silhouette_scores, marker='o')
plt.xlabel('Nombre de clusters')
plt.ylabel('Score de silhouette')
plt.title('Optimisation du nombre de clusters')
plt.show()

# Sélection et application du nombre optimal de clusters
optimal_clusters = np.argmax(silhouette_scores) + 2
kmeans_optimal = KMeans(n_clusters=optimal_clusters, random_state=42)
df_merged['cluster'] = kmeans_optimal.fit_predict(X_pca)

# À ce stade, vous pouvez continuer avec votre analyse de clusters ou avec une modélisation prédictive spécifique
# en fonction des clusters identifiés, comme illustré précédemment.

# Séparation des données en ensembles d'entraînement et de test
# X = df_merged.drop(columns=['cluster'])
# y = df_merged['cluster']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 
# # Entraînement d'un classificateur RandomForest
# rf = RandomForestClassifier(random_state=42)
# rf.fit(X_train, y_train)
# y_pred = rf.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy:.2f}')

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Temps d'exécution du script: {elapsed_time} secondes")


/var/folders/00/t_t93b0n0wx8m9fcms_xg6640000gn/T/ipykernel_74771/2337602903.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_demographie = pd.read_sql(query_demographie, conn)
/var/folders/00/t_t93b0n0wx8m9fcms_xg6640000gn/T/ipykernel_74771/2337602903.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_economie = pd.read_sql(query_economie, conn)
/var/folders/00/t_t93b0n0wx8m9fcms_xg6640000gn/T/ipykernel_74771/2337602903.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_election = pd.read_sql(query_election

KeyboardInterrupt: 